- [ ] Ler todos os arquivos
- [ ] Separar por municipios
- [ ] Aplicar funcao de limpeza
- [ ] Salvar no banco

In [1]:
PASTA = './../dados'

# Ler Arquivo

In [2]:
import pandas as pd

df = pd.read_parquet(f'{PASTA}/0.raw.parquet')

# Funçao suavizar

In [3]:
def suavizar(df, window_size=3, threshold=2):
    """
    Smooth data by replacing outliers with previous day's casosNovos.
    municipio, estado must be unique.
    
    Parameters:
    - df: pandas DataFrame with columns 'date' and 'casosNovos'
    - window_size: number of days to consider in rolling window
    - threshold: number of standard deviations to use for outlier detection
    
    Returns:
    - DataFrame with smoothed values in the column 'novos_casos_novos'
    """
    # Make a copy to avoid modifying original data
    df_smoothed = df.copy()

    df['novos_casos_novos'] = df['casosNovos']
    
    # Calculate rolling statistics
    rolling_mean = df['novos_casos_novos'].rolling(window=window_size, center=True, min_periods=1).mean()
    rolling_std = df['novos_casos_novos'].rolling(window=window_size, center=True, min_periods=1).std()
    
    # Identify outliers (values outside mean ± threshold*std)
    lower_bound = rolling_mean - threshold * rolling_std
    upper_bound = rolling_mean + threshold * rolling_std
    
    is_outlier = (df['novos_casos_novos'] < lower_bound) | (df['novos_casos_novos'] > upper_bound)
    
    # Replace outliers with previous day's value
    df_smoothed['novos_casos_novos'] = df['novos_casos_novos'].where(~is_outlier, df['novos_casos_novos'].shift(1))
    df_smoothed['novos_casos_novos'] = df['novos_casos_novos'].where(~is_outlier, df['novos_casos_novos'].shift(1))
    
    # For the first row (no previous value), use the next value if available
    if is_outlier.iloc[0] and len(df) > 1:
        df_smoothed.iloc[0, df_smoothed.columns.get_loc('novos_casos_novos')] = df['novos_casos_novos'].iloc[1]
    
    return df_smoothed

# Função remoção de outlier por z-score

In [4]:
from sklearn.preprocessing import StandardScaler

def remover_por_zscore(df):
    scaler = StandardScaler()
    LIMITE_COMUM = 1

    df["z_score"] = scaler.fit_transform(df[['novos_casos_novos']])
    df_sem_outliers = df[abs(df["z_score"]) < LIMITE_COMUM]

    return df_sem_outliers

# Recalcula casos acumulados

In [5]:
def recalcula_casos_acumulados(df):
  df['novos_casos_acumulados'] = df['novos_casos_novos'].cumsum()
  return df

# Função de limpeza

In [ ]:
def limpar(df):
  df = suavizar(df, window_size=5)
  df = remover_por_zscore(df)
  df = recalcula_casos_acumulados(df)
  return df

# Função principal - sincrona e csv

In [7]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [8]:
from tqdm import tqdm

def rodar(df):
    logging.info("Processando filtrar_regiao")
    df = df[df.regiao == "Sudeste"]
    logging.info("Processamento concluído")

    logging.info("Processando municipios_unicos")
    municipios_unicos = set(zip(df['municipio'], df['estado']))
    logging.info("Processamento concluído")
    
    logging.info("Filtrando o dataframe ")
    mascara = df.apply(lambda x: (x['municipio'], x['estado']) in municipios_unicos, axis=1)
    filtrado_df = df[mascara]
    logging.info("Processamento concluído")
    
    logging.info("Processando grupos")
    agrupado = filtrado_df.groupby(['municipio', 'estado'])
    logging.info("Processamento concluído")

    resultado_df = []
    
    for (municipio, estado), group_df in tqdm(agrupado, desc="Processando"):
        try:
            resultado_df.append(limpar(group_df))
        except Exception as e:
            logging.error(f"{estado}_{municipio} não foi salvo")
            logging.error(e)
    
    resultado_df = pd.concat(resultado_df)
    return resultado_df

In [9]:
processado_df = rodar(df)

2025-06-16 18:16:59 - INFO - Processando filtrar_regiao
2025-06-16 18:17:01 - INFO - Processamento concluído
2025-06-16 18:17:01 - INFO - Processando municipios_unicos
2025-06-16 18:17:01 - INFO - Processamento concluído
2025-06-16 18:17:01 - INFO - Filtrando o dataframe 
2025-06-16 18:17:33 - INFO - Processamento concluído
2025-06-16 18:17:33 - INFO - Processando grupos
2025-06-16 18:17:33 - INFO - Processamento concluído
Processando: 100%|█████████▉| 1668/1672 [00:16<00:00, 100.29it/s]


In [10]:
processado_df.to_parquet(f'{PASTA}/1.limpo.parquet', index=False)

In [11]:
data_set = set(zip(df['municipio'], df['estado']))

contains_none = any(None in pair for pair in data_set)
contains_none

True

In [12]:
none_tuples = [pair for pair in data_set if None in pair]
none_tuples

[(None, 'BA'),
 (None, 'PR'),
 (None, 'RR'),
 (None, 'PA'),
 (None, 'PB'),
 (None, 'SE'),
 (None, 'MS'),
 (None, 'RJ'),
 (None, 'MT'),
 (None, None),
 (None, 'PE'),
 (None, 'PI'),
 (None, 'MA'),
 (None, 'AP'),
 (None, 'SP'),
 (None, 'DF'),
 (None, 'RO'),
 (None, 'AM'),
 (None, 'CE'),
 (None, 'TO'),
 (None, 'GO'),
 (None, 'AL'),
 (None, 'RN'),
 (None, 'MG'),
 (None, 'AC'),
 (None, 'SC'),
 (None, 'RS'),
 (None, 'ES')]

In [13]:
nan_counts = df.isna().sum()
nan_counts

regiao                           0
estado                        1874
municipio                    91175
coduf                            0
codmun                       52472
codRegiaoSaude               91175
nomeRegiaoSaude              91175
data                             0
semanaEpi                        0
populacaoTCU2019             38703
casosAcumulado                   0
casosNovos                       0
obitosAcumulado                  0
obitosNovos                      0
Recuperadosnovos          10354813
emAcompanhamentoNovos     10354813
interior/metropolitana       91175
dtype: int64

In [14]:
df.apply(lambda x: (x['municipio'], x['estado']) in data_set, axis=1)

0           True
1           True
2           True
3           True
4           True
            ... 
10356680    True
10356681    True
10356682    True
10356683    True
10356684    True
Length: 10356685, dtype: bool